## Detekcia meteorov z obrázkov kamier AMOS pomocou hlbokého učenia

### Príprava pracovného prostredia pre spracovanie dát

Import potrebných knižníc.

In [1]:
import os
import csv
import pandas as pd
import numpy as np
import json
from PIL import Image, ImageDraw

## Spracovanie dát z anotačného projektu
### Vytvorenie tabuľky
Načítanie dát zo Zooniverse do premennej data.\
csvFile - názov `.csv` súboru, do ktorého sa tabuľka exportuje

In [2]:
data=pd.read_csv("meteor-v1-classifications.csv")
#data=pd.read_csv("meteor-v2-classifications.csv")
#data=pd.read_csv("meteor-satellite-classifications.csv")

csvFile="zooniverse-v1.csv"
#csvFile="zooniverse-v2.csv"
#csvFile="zooniverse-sat.csv"

In [7]:
with open(csvFile, mode='w', newline='') as csv_file:

    columns=["filename", "class", "x", "y", "width", "height"]
    write=csv.DictWriter(csv_file,fieldnames = columns)
    write.writeheader()
    i=0
    
    
    while (i<len(data)):
        
        is_meteor=json.loads(data.iloc[i,11])[0]["value"]
        
        if is_meteor == "Áno" or is_meteor =="Nie":
            metsat = 1
        else:
            metsat = 0
            
        if is_meteor == "Áno" or metsat == 0:
            
            anot=json.loads(data.iloc[i,11])[metsat]["value"]
            count=0
            
            if anot is not None:
                while(count<len(anot)):

                    file_name=json.loads(data.iloc[i,12])[str(data.iloc[i,13])]["Filename"]
                    x=json.loads(data.iloc[i,11])[metsat]["value"][count]["x"]
                    y=json.loads(data.iloc[i,11])[metsat]["value"][count]["y"]
                    width=json.loads(data.iloc[i,11])[metsat]["value"][count]["width"]
                    height=json.loads(data.iloc[i,11])[metsat]["value"][count]["height"]
                    eventClass=json.loads(data.iloc[i,11])[metsat]["value"][count]["tool_label"]

                    write.writerow({'filename': file_name,'class': eventClass.lower(),
                                    'x': x,'y':y,'width':width,'height':height})
                    count+=1
        i+=1

### Vykreslenie anotácií
rawPath - cesta ku surovým neupraveným obrázkom\
saveDest - názov priečinku, do ktorého sa upravené obrázky uložia. Ak zvolí používateľ rovnaký názov ako cestu ku obrázkom, pôvodné obrázky sa prepíšu\
csvFile - výstup z predošlého kroku

In [8]:
rawPath = r"rawdata" #path to raw images
saveDest = r"drawn_annotations" #save destination for the altered images, declare same as rawPath, if you want the raw images to be rewritten

csvFile = "zooniverse-v1.csv"
#csvFile = "zooniverse-v2.csv"
#csvFile = "zooniverse-sat.csv"

In [9]:
os.makedirs(saveDest, exist_ok=True)
data = pd.read_csv(csvFile)
#print(data)

for row in range(len(data)):
        file_name = data.at[row, 'filename']
        file_name = "1"+file_name
        objectClass = data.at[row, 'class']
        x = data.at[row, 'x']
        y = data.at[row, 'y']
        width = data.at[row, 'width']
        height = data.at[row, 'height']
        
        if objectClass == 'meteor':
            paint = "red"
        if objectClass == 'satellite':
            paint = "pink"
        
        path = os.path.join(rawPath, str(file_name))
        if os.path.exists(path) != True:
            continue
          
        alt_path = os.path.join(saveDest, str(file_name))
        if os.path.exists(alt_path) == True:
            path = alt_path
        
        im = Image.open(path)
        im1 = ImageDraw.Draw(im)
                                               
        shape=[(x, y), (x, y+height)]
        im1.line(shape, fill=paint, width=2)
        shape=[(x, y+height),(x+width,y+height) ]
        im1.line(shape, fill=paint, width=2)
        shape=[(x, y), (x+width, y)]
        im1.line(shape, fill=paint, width=2)
        shape=[(x+width, y),(x+width,y+height) ]
        im1.line(shape, fill=paint, width=2)
        im.save(os.path.join(saveDest, str(file_name)))

### Vytvorenie .txt súborov
Načítanie dát zo Zooniverse do premennej data

In [10]:
data=pd.read_csv("meteor-v1-classifications.csv")
#data=pd.read_csv("meteor-v2-classifications.csv")
#data=pd.read_csv("meteor-satellite-classifications.csv")

Vytvorenie priečinku `labels`, ak už vytvorený je, program využije existujúci priečinok

In [11]:
os.makedirs("labels", exist_ok=True)

Proces získania anotácie, určenie typu triedy a vytvorenie normalizovaného .txt súboru na intervale od 0 do 1

In [ ]:
textFile=os.getcwd()+"/labels"
i=0
while (i<len(data)):
        is_meteor=json.loads(data.iloc[i,11])[0]["value"]
        
        if is_meteor == "Áno" or is_meteor =="Nie":
            metsat = 1
        else:
            metsat = 0
            
        if is_meteor == "Áno" or metsat == 0:
            
            anot=json.loads(data.iloc[i,11])[metsat]["value"]
            event=json.loads(data.iloc[i,12])[str(data.iloc[i,13])]["Filename"].split(".")
            count=0
            
            with open(os.path.join(textFile, "%s.txt" % (event[0])), mode='w') as csv_file:
            
                while(count<len(anot)):

                    x=json.loads(data.iloc[i,11])[metsat]["value"][count]["x"]
                    y=json.loads(data.iloc[i,11])[metsat]["value"][count]["y"]
                    width=json.loads(data.iloc[i,11])[metsat]["value"][count]["width"]
                    height=json.loads(data.iloc[i,11])[metsat]["value"][count]["height"]
                    event_type=json.loads(data.iloc[i,11])[metsat]["value"][count]["tool_label"]

                    if(event_type == 'Meteor'):
                        label=0
                    elif (event_type == 'Satellite'):
                        label=1

                    fieldnames = [label,round((x+(width/2))/1280, 5),round((y+(height/2))/960, 5),round(width/1280, 5),round(height/960,5)]

                    writer = csv.DictWriter(csv_file,fieldnames, delimiter=' ')
                    writer.writeheader()
                    print("Just made normalized txt file for %s" % (event[0]))
                    count+=1
        i+=1